In [1]:
# manipulation des données
import numpy as np
import pandas as pd

# matplotlib et seaborn pour les représentations graphiques
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# sklearn preprocessing pour le traiter les variables catégorielles
from sklearn.preprocessing import LabelEncoder

# Gestion du système de fichiers
import os
from os import listdir

# Suppression des alertes
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../Data/5.augmented_dataset.csv')
data

,image_path,label
0,5eec66ec172ce096cd4e5c947686b56a.jpg,0
1,ffc2e1fc7c886ec09a331040ada038aa.jpg,4
2,AI6c3768639e82ceaeab681e083b897672.jpg,2
3,AId98fac46793639eceaaa2e74cadd4973.jpg,6
4,6e812fbc03331ab363789b0f2d6b3ff2.jpg,0
...,...,...
2095,AIa3dea07b2f7bf48276d32513091be717.jpg,5
2096,AId47f1cb78db4a23e2516e2d34da34c8a.jpg,5
2097,AI1afa5f2c3c9b4b43def6dc2e695d2c87.jpg,5
2098,AIace154420a51fad090b3543995630051.jpg,4


In [3]:
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
from glob import glob

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalAveragePooling1D, Flatten, Dense, Dropout 
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

# os.environ["TF_KERAS"]='1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-01-03 10:51:37.832432: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  0


In [4]:
base_model = VGG16()
model = Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
path = '../Data/AugmentedImages/'

def preprocess_image(image_file):
    image = load_img(path+image_file, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

X = data.drop('label', axis=1).values
y = data['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

image = preprocess_image(X_train)

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))


images_features = []
images_features.append(model.predict(image, verbose=0)[0]) # predict from pretrained model

images_features = np.asarray(images_features)

In [17]:
from keras.utils import to_categorical
path = '../Data/AugmentedImages/'

base_model = VGG16()
model = Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)

def preprocess_image(image_file):
    image = load_img(path+image_file, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

X = np.array([preprocess_image(image_file) for image_file in data['image_path']])
y = to_categorical(data['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

base_model = VGG16()
model = Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10


ValueError: in user code:

    File "/Applications/anaconda3/envs/Projet6Env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Applications/anaconda3/envs/Projet6Env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Applications/anaconda3/envs/Projet6Env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Applications/anaconda3/envs/Projet6Env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Applications/anaconda3/envs/Projet6Env/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Applications

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical

# Charger le dataframe avec les noms de fichiers d'images et les labels
# Supposons que le dataframe a deux colonnes : 'filename' et 'label'
df = pd.read_csv('../Data/5.augmented_dataset.csv')
path = '../Data/AugmentedImages/'

# Prétraiter les images
def preprocess_image(img_path):
    img = image.load_img(path + img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Charger les images et les labels
X = np.vstack([preprocess_image(filename) for filename in df['image_path']])
y = to_categorical(df['label'])

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Charger le modèle VGG16 sans la couche Dense supérieure (include_top=False)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Gel des poids des couches VGG16 pré-entraînées
for layer in base_model.layers:
    layer.trainable = False

# Créer le modèle complet
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(7, activation='softmax'))  # Assurez-vous de définir correctement le nombre de classes

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
53/53 [==============================] - 302s 6s/step - loss: 11.2249 - accuracy: 0.7119 - val_loss: 2.0667 - val_accuracy: 0.7976
Epoch 2/10
53/53 [==============================] - 292s 6s/step - loss: 0.2670 - accuracy: 0.9506 - val_loss: 1.5312 - val_accuracy: 0.8452
Epoch 3/10
53/53 [==============================] - 294s 6s/step - loss: 0.0912 - accuracy: 0.9839 - val_loss: 1.3341 - val_accuracy: 0.8619
Epoch 4/10
53/53 [==============================] - 293s 6s/step - loss: 0.0127 - accuracy: 0.9970 - val_loss: 1.1919 - val_accuracy: 0.8690
Epoch 5/10
53/53 [==============================] - 292s 6s/step - loss: 0.0063 - accuracy: 0.9982 - val_loss: 1.2971 - val_accuracy: 0.8690
Epoch 6/10
53/53 [==============================] - 291s 6s/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 1.1930 - val_accuracy: 0.8810
Epoch 7/10
53/53 [==============================] - 291s 6s/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 1.1893 - val_accuracy: 0.8786
Epoch 8/10
5

In [24]:
score = model.evaluate(X_test, y_test, verbose=0)

print('Test loss    :', score[0])
print('Test accuracy:', score[1])# mean absolute error ?????

Test loss    : 1.1938369274139404
Test accuracy: 0.8761904835700989
